In [97]:
import pathlib
import superintendent
import ipywidgets as widgets
from ipywidgets import GridspecLayout
from IPython.display import display
import numpy as np
from skimage.io import imread
import matplotlib.pyplot as plt
from PIL import Image
import io
import math

images = [f for f in pathlib.Path("/home/jovyan/data/chips").glob("*/img_ndwi*.png") if f.is_file()]
print(len(images))

def change_colormap(image_path:pathlib.Path, cmap='RdYlBu'):
    feature = imread(image_path)
    cm = plt.get_cmap(cmap)
    colored_image = cm(-feature)
    colored_feature = Image.fromarray(np.uint8(colored_image * 255))
    imgByteArr = io.BytesIO()
    colored_feature.save(imgByteArr, format='PNG')
    imgByteArr = imgByteArr.getvalue()
    return imgByteArr
    
def display_image_and_references(image_path):
    image_folder = image_path.parent
    
    other_images = [
        f for f in image_folder.glob("img_ndwi*.png")
        if f.is_file() and f != image_path
    ]
   
    n_col = 3
    n_row = max(math.ceil(len(other_images) / n_col), 1)
    grid = GridspecLayout(n_row, n_col)
    for i in range(n_row):
        for j in range(n_col):
            img_index = j * n_col + i
            if img_index >= len(other_images): break
            image = other_images[img_index]
            grid[i, j] = widgets.VBox([
                    widgets.Label(f"Image {image.name}"),
                    widgets.Image(value=change_colormap(image), 
                             layout=widgets.Layout(width='200px', height='200px')),
                ])
    
    image_display = widgets.VBox([
        widgets.VBox([
            widgets.Label(f"folder: {image_folder}"),
            widgets.Label(f"all other images of the same loc"),
            grid,
            widgets.Label(f"image to label: {image_path.name}"),
            widgets.Image(value=change_colormap(image_path), object_fit='none', 
                          layout=widgets.Layout(width='300px', height='300px'))
        ]),
        
    ])
    display(image_display)

labeller = superintendent.ClassLabeller(
    features=images,
    options=[i for i in range(6)],
    display_func=display_image_and_references,
)

labeller

499


ClassLabeller(children=(HBox(children=(FloatProgress(value=0.0, description='Progress:', max=1.0),)), Box(chil…